# Adding memory

This shows how to add memory to an arbitrary chain. Right now, you can use the memory classes but need to hook it up manually

In [1]:
from operator import itemgetter
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

In [2]:
memory = ConversationBufferMemory(return_messages=True)

In [3]:
memory.load_memory_variables({})

{'history': []}

In [4]:
chain = (
    RunnablePassthrough.assign(
        memory=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
    )
    | prompt
    | model
)

In [5]:
inputs = {"input": "hi im bob"}
response = chain.invoke(inputs)
response

AIMessage(content='Hello Bob! How can I assist you today?', additional_kwargs={}, example=False)

In [6]:
memory.save_context(inputs, {"output": response.content})

In [7]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='hi im bob', additional_kwargs={}, example=False),
  AIMessage(content='Hello Bob! How can I assist you today?', additional_kwargs={}, example=False)]}

In [8]:
inputs = {"input": "whats my name"}
response = chain.invoke(inputs)
response

AIMessage(content='Your name is Bob.', additional_kwargs={}, example=False)